In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### CNN implementation with use of GPU power

In [ ]:
df = pd.read_csv('../input/digit-recognizer/train.csv')

In [ ]:
df.describe()

## Extract X and y with normalization of X

In [ ]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]
X/=255

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf


## Because large amount of data, test_size percentage is lowered

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.03, stratify=y)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

## One-hot for y (10-digits)

In [ ]:
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values

## Preparing input for CNN

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
input_shape = X_train.shape[1:]

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if logs.get('val_loss') < 0.007:
            print("\nReached desired loss")
            self.model.stop_training = True

callbacks = myCallback()

### 3 Conv2D layers without with 2 Pooling layers. (BN Dropout + Data Augmentation)

In [ ]:
X_input = Input(input_shape)

X = Conv2D(24, (5, 5), strides = (1, 1), name = 'conv0')(X_input)
X = BatchNormalization(axis = -1, name = 'bn1')(X)
X = Activation('relu')(X)
X = Dropout(0.2)(X)
X = MaxPooling2D((2, 2), name='max_pool')(X)
X = Conv2D(48, (3, 3), strides = (1, 1), name = 'conv1')(X)
X = BatchNormalization(axis = -1, name = 'bn2')(X)
X = Activation('relu')(X)
X = Dropout(0.2)(X)
X = MaxPooling2D((2, 2), name='max_pool2')(X)
X = Conv2D(96, (3, 3), strides = (1, 1), padding='same',name = 'conv2')(X)
X = BatchNormalization(axis = -1, name = 'bn2_2')(X)
X = Activation('relu')(X)
X = Dropout(0.2)(X)
X = Flatten()(X)

X = Dense(128, name = 'lay3')(X)
X = BatchNormalization(axis = -1, name = 'bn3')(X)
X = Activation('relu')(X)


X = Dense(10, activation='softmax', name='output')(X)
model = Model(inputs = X_input, outputs = X, name='MnistDigit')

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

### Data Augmentation step

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=15,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

datagen.fit(X_train)
#datagen = keras.preprocessing.image.ImageDataGenerator()
#datagen =  datagen.fit(X_train)

In [ ]:
#model.fit(x=X_train, y = y_train, validation_data=(X_test,y_test), epochs = 450, batch_size=128, callbacks=[callbacks])

model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
          epochs=100,validation_data=(X_test,y_test), callbacks=[callbacks])

In [ ]:
df_test = pd.read_csv('../input/digit-recognizer/test.csv')
X_pred = np.array(df_test)
X_pred = X_pred/255.0

In [ ]:
X_pred = X_pred.reshape((X_pred.shape[0], 28, 28, 1))
y_pred = model.predict(X_pred)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred[:5]

In [ ]:
result = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
result['Label'] = y_pred
result.head()

In [ ]:
result.to_csv('result.csv', index=False)